In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housex-text-processing/housex-processing/selected_artists.txt
/kaggle/input/housex-text-processing/housex-processing/corpus/melodic_house/1.txt
/kaggle/input/housex-text-processing/housex-processing/corpus/melodic_house/2.txt
/kaggle/input/housex-text-processing/housex-processing/corpus/future_house/3.txt
/kaggle/input/housex-text-processing/housex-processing/corpus/future_house/1.txt
/kaggle/input/housex-text-processing/housex-processing/corpus/future_house/2.txt
/kaggle/input/housex-text-processing/housex-processing/corpus/future_house/4.txt
/kaggle/input/housex-text-processing/housex-processing/corpus/progressive_house/3.txt
/kaggle/input/housex-text-processing/housex-processing/corpus/progressive_house/1.txt
/kaggle/input/housex-text-processing/housex-processing/corpus/progressive_house/2.txt
/kaggle/input/housex-text-processing/housex-processing/corpus/progressive_house/4.txt
/kaggle/input/housex-text-processing/housex-processing/corpus/bass_house/3.txt
/kaggle/input

## Load descriptors for a file

In [2]:
def get_file_descriptors(file_path):
    try:
        with open(file_path, "r") as f:
            lines = f.readlines()
            lines = lines[1:] # Skip the URL
            lines = [x for x in lines if len(x) > 0] # Skip empty lines
            merged_string = ' '.join(lines)
            return merged_string
    except Exception as e:
        print(f"Error: {e}")
        return None

In [3]:
get_file_descriptors("/kaggle/input/housex-text-processing/housex-processing/corpus/melodic_house/1.txt")

'The recently released EP from Avicii titled “Avici” has not only got some amazing responses but some great remixes too. One of such fine remixes is from the Norwegian producer Jack Taylor. Giving an emotional progressive twist to the original song, the remix stands out and delivers some great vibes. The young artist followed the Nort European trend of making beautiful and emotional progressive, and once again I’m proud to announce a new talent to check out!\n The remix starts out with a energetic intro, including the lovely guitar snippet from the original track. Smoothly guiding into the breakdown, the original track comes up. Just as the break from the original track changes to the drop, an charming guitar melody comes up…. It really feels like a piece of the previous version and blends in perfectly. An example of good transition!\n The drop feels epic with the same melody, now backed with some more saw synths and the same guitar melody. I really have to admire Jack’s idea of extend

## Store the descriptors for each sub-genre

In [5]:
from glob import glob
all_files = glob('/kaggle/input/housex-text-processing/housex-processing/corpus' + '/**/*.txt', recursive=True)

GENRE_DICT = {}

for current_file in all_files:
    genre = current_file.split('/')[-2]
    if genre not in GENRE_DICT:
        GENRE_DICT[genre] = []
    desc = get_file_descriptors(current_file)
    GENRE_DICT[genre].append(desc)

In [6]:
GENRE_DICT.keys()

dict_keys(['melodic_house', 'future_house', 'progressive_house', 'bass_house'])

## Load Model from Huggingface
- stablelm-base-alpha-3b-v2

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-base-alpha-3b-v2")
model = AutoModelForCausalLM.from_pretrained(
  "stabilityai/stablelm-base-alpha-3b-v2",
  trust_remote_code=True,
  torch_dtype="auto",
)
model.cuda()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

StableLMAlphaForCausalLM(
  (transformer): StableLMAlphaModel(
    (embed): Embedding(50432, 2560)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attention): Attention(
          (qkv_proj): Linear(in_features=2560, out_features=7680, bias=False)
          (out_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=2560, out_features=13824, bias=False)
          (out_proj): Linear(in_features=6912, out_features=2560, bias=False)
          (act): SiLU()
        )
      )
    )
    (final_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=2560, out_features=50432, bias=False)
)

## Sample StableLM run

In [8]:
inputs = tokenizer("The weather is always wonderful", return_tensors="pt").to("cuda")
tokens = model.generate(
  **inputs,
  max_new_tokens=100,
  temperature=0.95,
  top_p=0.95,
  do_sample=True,
)
print(tokenizer.decode(tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


The weather is always wonderful here, and a short drive takes you to the mountains which is a great place to escape for a few days. The resort has a good location, a clean pool and comfortable rooms. The staff is always very friendly and helpful. We will definitely stay again. more


## Prompting strategy

In [9]:
from tqdm import tqdm
PROMPT_DICT_1 = {}

for _k in GENRE_DICT:
    if _k not in PROMPT_DICT_1:
        PROMPT_DICT_1[_k] = []
    # Get the output from FLAN-T5-base
    for inputs in tqdm(GENRE_DICT[_k]):
        prompt = f"Generate 5 verbs to summarise the house music piece described as follows: '{inputs}'. Output only 5 words separated by ','. Note that this is a {_k} song. The selected words should be consistent with the input text. Do not output full sentences."
        print('='*40)
        print(f"Prompt: {prompt}")
        print('='*40)
        inputs = tokenizer(prompt, return_tensors="pt")
        inputs.to("cuda")
        outputs = model.generate(
          **inputs,
          max_new_tokens=100,
          temperature=0.95,
          top_p=0.95,
          do_sample=True,
        )
        selected_words = list(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))[0]
        PROMPT_DICT_1[_k].append(selected_words)

  0%|          | 0/2 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt: Generate 5 verbs to summarise the house music piece described as follows: 'The recently released EP from Avicii titled “Avici” has not only got some amazing responses but some great remixes too. One of such fine remixes is from the Norwegian producer Jack Taylor. Giving an emotional progressive twist to the original song, the remix stands out and delivers some great vibes. The young artist followed the Nort European trend of making beautiful and emotional progressive, and once again I’m proud to announce a new talent to check out!
 The remix starts out with a energetic intro, including the lovely guitar snippet from the original track. Smoothly guiding into the breakdown, the original track comes up. Just as the break from the original track changes to the drop, an charming guitar melody comes up…. It really feels like a piece of the previous version and blends in perfectly. An example of good transition!
 The drop feels epic with the same melody, now backed with some more saw 

 50%|█████     | 1/2 [00:03<00:03,  3.32s/it]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt: Generate 5 verbs to summarise the house music piece described as follows: 'I wanted to end this beautiful September with a masterpiece. Steerner, one of the most underrated producers in the history, is back with a monstrous “More Than You Know” bootleg.
 “More Than You Know” (Steerner Bootleg) impressed me, using a euphemism. This is how a bootleg should be made, with the exact balance between original vibes, experimentation and signature sounds. 33% each one, and the remaining 1% goes in goosebumps.
 Talking about the original version, I’ve never reviewed it. I wasn’t a big fan of it, I liked the vocal but not the slow rhythm, During the past months some unofficial versions came out (my favorite was the Firebeatz one), but I had not found the “right remix”. There was potential, obviously, we are talking about A&I, the song is good, but not 100% good.
 I know it’s quite difficult to understand, but in few words, I was looking for the right “More Than You Know”, and it arrived t

  0%|          | 0/4 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

 50%|█████     | 2/4 [00:04<00:04,  2.44s/it]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt: Generate 5 verbs to summarise the house music piece described as follows: '• Creative sound arrangements
 • Blend of Bass and Tech House
 • Minimal but aggressive
 With the renowned Dutch label of STMPD pushing out their own flavour of House music with the help of their multi-talented roster. Among this coveted list of musicians, the Dutch personage Loopers has made a steady and unhindered reputation in Martin Garrix’s label since the last couple of years. This Friday witnessed his latest release, “Groove Police”, a type of production which strikes a chord within a few seconds of its playback, emitting an energetic and groovy vibe meant for accelerating the after-hours dance-floor activity.
 Loopers has reclaimed his place in STMPD with flair, not to forget his collaboration with native heavyweights Martin Garrix & Dyro. The preceding year, the esteemed solo act made explosive dance numbers such as “Out Of Control” (with Seth Hills), extrapolating and experimenting with the upr

 75%|███████▌  | 3/4 [00:04<00:01,  1.53s/it]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt: Generate 5 verbs to summarise the house music piece described as follows: '• Clear superhero soundtrack theme
 • Surprising vocal performance
 • Right balance of orchestral and EDM
 After the promising feedback on “Sentio“, I was craving for a final Garrix track before the dusk of the year, and lucky me: “Hero” revealed itself! I am impressed by the versatility shown by the Dutch creator, as this time he takes a trip into the pop and commercial realms. Surprise surprise, he’s no stranger to that and we won’t be bashing as there’s a great deal to unravel here!
 First of all, “Hero” is released in collaboration with Marvel’s “Snap” mobile game. While I am particularly devoted to the game’s premise itself (usually card games are riddled with micro-transactions), it clearly explains the title, design, and visuals (by the way, the music video is sick!). For Marvel’s PR team, this is a solid plan! The vocalist is JVKE, whom I didn’t originally prefer as he is principally a TikTok inf

100%|██████████| 4/4 [00:05<00:00,  1.49s/it]


In [10]:
PROMPT_DICT_1

{'melodic_house': ['Generate', 'Generate'],
 'future_house': ['Generate', 'Generate', 'Generate', 'Generate'],
 'progressive_house': ['Generate', 'Generate', 'Generate', 'Generate'],
 'bass_house': ['Generate', 'Generate', 'Generate', 'Generate']}

## Generate `.md` file for the outputs

In [11]:
for _k in PROMPT_DICT_1:
    print(_k)
    for desc in PROMPT_DICT_1[_k]:
        print(f" - {desc}")
    print()

melodic_house
 - Generate
 - Generate

future_house
 - Generate
 - Generate
 - Generate
 - Generate

progressive_house
 - Generate
 - Generate
 - Generate
 - Generate

bass_house
 - Generate
 - Generate
 - Generate
 - Generate

